In [109]:
import pandas as pd
import re
from datetime import datetime

In [110]:
df = pd.read_csv("orders.csv")
df_ab = pd.read_csv('abandoned.csv')

df.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
0,#195763,jasonstuart930@gmail.com,paid,2023-02-15 08:24:52 -0800,fulfilled,2023-02-15 10:46:42 -0800,yes,USD,235.94,22.76,...,NaN,NaN,NaN,NaN,Kentucky,Kentucky,c27962692075603.1,NaN,NaN,c27962692075603.1
1,#195763,jasonstuart930@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#195764,brian@rossmgt.com,paid,2023-02-15 08:40:31 -0800,fulfilled,2023-02-15 10:54:12 -0800,yes,USD,346.43,35.53,...,NaN,NaN,NaN,NaN,Washington,Washington,c27958684188755.1,NaN,NaN,c27958684188755.1
3,#195764,brian@rossmgt.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#195764,brian@rossmgt.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# make order table

In [125]:
def Make_Order_Table(df):
    order = df[
        [    'Name','Created at','Financial Status',
            'Paid at', 'Fulfillment Status', 
             'Fulfilled at','Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
             'Taxes','Total', 'Discount Code', 'Discount Amount', 'Shipping Method',
             'Billing Name', 'Billing Address1', 'Billing Address2', 
             'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country',
             'Billing Phone', 'Shipping Name', 'Shipping Address1', 
             'Shipping Address2', 'Shipping City', 'Shipping Zip',
             'Shipping Province', 'Shipping Country', 'Shipping Phone', 'Notes','Payment Method'
    ]
    ].copy()

    #Remove duplicate values
    order['Name'] = order['Name'].str.replace("#", "")
    order['Name'] = order["Name"].drop_duplicates()
    order = order[order["Name"].notna()]
    
    #Map Proper names
    order.rename(columns =
                    {
                        'Name':'order_id',
                        'Created at':'created_at',
                        'Financial Status':'financial_status',
                        'Paid at':'paid_at',
                        'Fulfillment Status':'fulfillment_status', 
                        'Fulfilled at':'fulfilled_at',
                        'Accepts Marketing':'accepts_marketing',
                        'Currency':'currency',
                        'Subtotal':'subtotal',
                        'Shipping':'shipping',
                        'Taxes':'taxes',
                        'Total':'total',
                        'Discount Code':'discount_code',
                        'Discount Amount':'discount_amount',
                        'Shipping Method':'shipping_method',
                        'Billing Name': 'billing_name',
                        'Billing Address1':'billing_address1',
                        'Billing Address2':'billing_address2',
                        'Billing City':'billing_city',
                        'Billing Zip':'billing_zip',
                        'Billing Province':'billing_province',
                        'Billing Country': 'billing_country',
                        'Billing Phone':'billing_phone',
                        'Shipping Name':'shipping_name',
                        'Shipping Address1' : 'shipping_address1',
                        'Shipping Address2' : 'shipping_address2',
                        'Shipping City':'shipping_city',
                        'Shipping Zip':'shipping_zip',
                        'Shipping Province':'shipping_province',
                        'Shipping Country':'shipping_country',
                        'Shipping Phone':'shipping_phone', 
                        'Notes':'notes',
                        'Payment Method':'payment_method',    
                    }, 
                    
                    inplace = True
                   )
    order.reset_index(drop = True, inplace = True)

    # Convert to datetime object and remove timezone info
    order['created_at'] = pd.to_datetime(order['created_at']).dt.tz_localize(None)
    order['paid_at'] = pd.to_datetime(order['paid_at']).dt.tz_localize(None)
    order['fulfilled_at'] = pd.to_datetime(order['paid_at']).dt.tz_localize(None)
    
    # Format as string in desired format
    order['created_at'] = order['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
    order['paid_at'] = order['paid_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
    order['fulfilled_at'] = order['fulfilled_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Fill NaN values
    order['created_at'] = order['created_at'].fillna('2020-01-01 00:00:00')
    order['paid_at'] = order['paid_at'].fillna('2020-01-01 00:00:00')
    order['fulfilled_at'] = order['fulfilled_at'].fillna('2020-01-01 00:00:00')
    order.fillna('null', inplace = True)
    
    return order

# Customer Table

In [126]:
def Make_Customer_Table(df):
    customer = df[
        ['Email', 'Billing Name', 'Billing Province']
    ].copy()
    
    customer['Email'] = customer["Email"].drop_duplicates()
    customer = customer[customer["Email"].notna()]
    customer.rename(columns = {
                            'Email':'email',
                            'Billing Name':'full_name',
                            'Billing Province':'location'
                    },                    
                    inplace=True
                   )
    
    customer.fillna('null', inplace = True)
   
    customer.reset_index(drop=True, inplace=True)   
    return customer

# Product Table

In [136]:
def Make_Product_Table(df):
    product = df[
        ['Lineitem sku', 'Lineitem name', 'Lineitem price',
         'Lineitem compare at price','Lineitem requires shipping',
         'Lineitem taxable']
    ].copy()
    
    product['Lineitem name'] = product["Lineitem name"].drop_duplicates()
    product = product[product["Lineitem name"].notna()]
    product['Lineitem price'] = product["Lineitem price"].drop_duplicates()
   
    product = product[product["Lineitem price"].notna()]   
    product.fillna('null', inplace = True)

    #Map Proper names
    product.rename(columns =
                    {
                        'Lineitem sku':'product_sku',
                        'Lineitem name':'product_name',
                        'Lineitem price':'product_price',
                        'Lineitem compare at price':'product_compare_at_price',
                        'Lineitem requires shipping':'product_requires_shipping', 
                        'Lineitem taxable':'product_taxable',                      
                    },
                   
                   inplace = True
                  )  
    
    product.reset_index(drop = True, inplace = True)
        
    return product

# Order Item

In [128]:
def Make_Order_Item_Table(df):
    order_item = df[
        ['Name', 'Lineitem sku', 'Lineitem quantity', 'Lineitem fulfillment status']
    ].copy()
    
    order_item['Name'] = order_item['Name'].str.replace("#", "")
    
    order_item.fillna('null', inplace = True)
    
    order_item.reset_index(drop = True, inplace = True)
    
    return order_item


In [137]:
abandoned_order = Make_Order_Table(df_ab)
# abandoned_order.drop(['Financial Status', 'Paid at', 'Fulfillment Status'], axis=1)

abandoned_order_item = Make_Order_Item_Table(df_ab)

order_item = Make_Order_Item_Table(df)
product = Make_Product_Table(df)
customer = Make_Customer_Table(df)
order = Make_Order_Table(df)

C:\Users\armen\anaconda3\lib\site-packages\pandas\core\indexes\extension.py:101: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(result, name=self.name)


# Transfer to DB

In [142]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(user='root', password='arm2002',
                        host='127.0.0.1', database='coolina',
                        auth_plugin='mysql_native_password')    
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        
        # Insert into customer
        
        n = 1
        
        for i,row in customer.iterrows():
            sql = ("INSERT IGNORE INTO coolina.customer "
                   "(email, full_name, location) "
                   "VALUES (%s, %s, %s)"
                  )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted Customer")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1
            
        
        # Insert into orders
        
        n = 1
        
        for i,row in order.iterrows():
            sql = ("INSERT IGNORE INTO coolina.orders "
                   "(order_id, created_at, financial_status, paid_at,"
               "fulfillment_status, fulfilled_at, accepts_marketing, currency,"
               "subtotal, shipping, taxes, total, discount_code,"
               "discount_amount, shipping_method, billing_name,"
               "billing_address1, billing_address2, billing_city, billing_zip,"
               "billing_province, billing_country, billing_phone, shipping_name,"
               "shipping_address1, shipping_address2, shipping_city,"
               "shipping_zip, shipping_province, shipping_country,"
               "shipping_phone, notes, payment_method)"
                   "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,"
                   " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                   )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted to ORDERS")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1            
            
            
         
        # Insert into product
        n = 1
        
        for i,row in product.iterrows():
            sql = ("INSERT IGNORE INTO coolina.product "
            "(product_sku, product_name, product_price, product_compare_at_price, product_requires_shipping, product_taxable) "
            "VALUES (%s, %s, %s, %s, %s, %s)"
             )
            cursor.execute(sql, tuple(row))
            print(f"{n} Record inserted Product")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
            n += 1

except Error as e:
    print("Error while connecting to MySQL", e)

You're connected to database:  ('coolina',)
1 Record inserted Customer
2 Record inserted Customer
3 Record inserted Customer
4 Record inserted Customer
5 Record inserted Customer
6 Record inserted Customer
7 Record inserted Customer
8 Record inserted Customer
9 Record inserted Customer
10 Record inserted Customer
11 Record inserted Customer
12 Record inserted Customer
13 Record inserted Customer
14 Record inserted Customer
15 Record inserted Customer
16 Record inserted Customer
17 Record inserted Customer
18 Record inserted Customer
19 Record inserted Customer
20 Record inserted Customer
21 Record inserted Customer
22 Record inserted Customer
23 Record inserted Customer
24 Record inserted Customer
25 Record inserted Customer
26 Record inserted Customer
27 Record inserted Customer
28 Record inserted Customer
29 Record inserted Customer
30 Record inserted Customer
31 Record inserted Customer
32 Record inserted Customer
33 Record inserted Customer
34 Record inserted Customer
35 Record ins

# INSERT Product

In [138]:
product   

,product_sku,product_name,product_price,product_compare_at_price,product_requires_shipping,product_taxable
0,AKHC110C,Machado Butcher Knife,109.97,275.99,True,True
1,AKHC102C,Altomino Handmade Chef Knife,125.97,189.99,True,True
2,AKHC108C,Altomino Handmade Butcher Knife,104.97,179.99,True,True
3,ECIK03,Custom engraving,15.00,null,True,False
4,AKHC104C,Altomino Tungsten Steel Slicing Knife,157.97,198.99,True,True
5,AKHC101C,Promaja,99.97,215.99,True,True
6,AXWS401C,Kezuriki - Premium Whetstone,69.97,119.98,True,True
7,AKHC103C,LIXY - Hand-Forged Cleaver Knife,199.97,349.99,True,True
8,ASPU305C,Altomino Handmade Fixed Blade Sharp Knife's PU...,31.97,59.99,True,True
9,ASPU304C,LIXY Knife's PU Sheath,36.97,59.99,True,True


1 Record inserted Product
2 Record inserted Product
3 Record inserted Product
4 Record inserted Product
5 Record inserted Product
6 Record inserted Product
7 Record inserted Product
8 Record inserted Product
9 Record inserted Product
10 Record inserted Product
11 Record inserted Product
12 Record inserted Product
13 Record inserted Product
14 Record inserted Product
15 Record inserted Product
16 Record inserted Product
17 Record inserted Product
18 Record inserted Product


In [ ]:
#            "(order_id, created_at, financial_status, paid_at,"
#        "fulfillment_status, fulfilled_at, accepts_marketing, currency,"
#        "subtotal, shipping, taxes, total, discount_code,"
#        "discount_amount, shipping_method, billing_name,"
#        "billing_address1, billing_address2, billing_city, billing_zip,"
#        "billing_province, billing_country, billing_phone, shipping_name,"
#        "shipping_address1, shipping_address2, shipping_city,"
#        "shipping_zip, shipping_province, shipping_country,"
#        "shipping_phone, notes, payment_method)"

In [141]:
product

,product_sku,product_name,product_price,product_compare_at_price,product_requires_shipping,product_taxable
0,AKHC110C,Machado Butcher Knife,109.97,275.99,True,True
1,AKHC102C,Altomino Handmade Chef Knife,125.97,189.99,True,True
2,AKHC108C,Altomino Handmade Butcher Knife,104.97,179.99,True,True
3,ECIK03,Custom engraving,15.00,null,True,False
4,AKHC104C,Altomino Tungsten Steel Slicing Knife,157.97,198.99,True,True
5,AKHC101C,Promaja,99.97,215.99,True,True
6,AXWS401C,Kezuriki - Premium Whetstone,69.97,119.98,True,True
7,AKHC103C,LIXY - Hand-Forged Cleaver Knife,199.97,349.99,True,True
8,ASPU305C,Altomino Handmade Fixed Blade Sharp Knife's PU...,31.97,59.99,True,True
9,ASPU304C,LIXY Knife's PU Sheath,36.97,59.99,True,True
